In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

mdata = pd.read_csv('/home/mickey/Documents/CSV/mal.csv', encoding = 'big5')
x =[]
label = pd.read_csv('/media/dataset/dataset.csv', encoding = 'big5', dtype={'Column2': str})
label_x=[]
label_y=[]
for data in mdata['feature']:
    label_x.append(" ".join(data[i:i+2] for i in range(0, len(data), 2)))

vectorizer = CountVectorizer(analyzer='word', ngram_range=(4, 4), lowercase=False)
X=vectorizer.fit_transform(label_x)
start=0
for data in mdata['file']:
    count=0
    f=data[0]+data[1]
    if(start==-1):
        for infor in label['filename']:
            if(infor[0]+infor[1]!=f):
                start=start+1
            else:
                break

        while(label['filename'][start+count]!=data):
            count=count+1
        label_y.append(label['label'][start+count])
    elif(label['filename'][start][0]+label['filename'][start][1]!=f):
        start=0
        for infor in label['filename']:
            if(infor[0]+infor[1]!=f):
                start=start+1
            else:
                break

        while(label['filename'][start+count]!=data):
            count=count+1
        label_y.append(label['label'][start+count])
    else:
        while(label['filename'][start+count]!=data):
            count=count+1
        label_y.append(label['label'][start+count])
        
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
svm_classifier = SGDClassifier(loss='hinge', alpha=1/0.2)
knn_classifier = KNeighborsClassifier(n_neighbors=3)
nb_classifier =  MultinomialNB()
mlp_classifier = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=1000)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,label_y,test_size = 0.2,shuffle = True)
svm_classifier.fit(x_train,y_train)
print("SVM:",accuracy_score(y_test,svm_classifier.predict(x_test)))


/tmp/ipykernel_414762/4067651784.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  label = pd.read_csv('/media/dataset/dataset.csv', encoding = 'big5', dtype={'Column2': str})


In [ ]:
confusion_matrix(y_test, svm_classifier.predict(x_test))

In [17]:
x=5

In [59]:
import numpy as np
import sys
import pandas as pd
import os
import r2pipe
import pickle

def extractFeature(data_path):
    datalist=[]
    for file in os.listdir(data_path):         
        filepath=data_path+'/'+file  
        
        r2 = r2pipe.open(filepath)
        r2.cmd('aaa')
        r2.cmd(f's {"main"}')
        code = r2.cmd('pd')
        r2.quit()
        feature = ""
        i = 0
            
        l=len(code.split('\n'))
        for i in range(l):
            temp = [_str for _str in code.split('\n')[i].split(' ') if _str != '']
            for j in range(len(temp)):
                try:
                    if temp[j][0:2]=="0x" and all(temp[j+1][k].isdigit() or temp[j+1][k] in 'abcdef' for k in range(8)):
                        feature+=''.join(temp[j+1])
                        break
                except IndexError:
                    break
   
        feature = feature.ljust(2048, '0')[:2048]
        datalist.append({'file':file,'feature':feature}) 
    
    
    with open('data.pickle', 'wb') as file:
        pickle.dump(datalist, file)
            

In [60]:
extractFeature('/media/dataset/linuxmal/f8/')

1

In [57]:
import pickle
import sys
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
def vectorize():
    with open('data.pickle', 'rb') as file:
        loaded_data = pickle.load(file)
    label = pd.read_csv('/media/dataset/dataset.csv', encoding = 'big5', dtype={'Column2': str})
    label_x=[]
    label_y=[]
    for data in [record['feature'] for record in loaded_data]:
        label_x.append(" ".join(data[i:i+2] for i in range(0, len(data), 2)))

    vectorizer = CountVectorizer(analyzer='word', ngram_range=(4, 4), lowercase=False)
    X=vectorizer.fit_transform(label_x)
    start = 0
    
    for data in [record['file'] for record in loaded_data]:
        print(data)
        count = 0
        f = data[0] + data[1]

        while start < len(label['filename']) and label['filename'][start][0] + label['filename'][start][1] != f:
            start += 1

        while label['filename'][start + count] != data:
            count += 1

        label_y.append(label['label'][start + count])

    return X, label_y

In [65]:
with open('data.pickle', 'rb') as file:
    loaded_data = pickle.load(file)
print(type(loaded_data))
print(loaded_data)

<class 'list'>
[{'file': 'f817e8a97727abcdd368703e3fcce2f76b773c54b7a9efe11d7330d7c2f6ad75', 'feature': '9de3b0009c03bef8f027a044f227a048030000c182106170c227bfbc030000c1901061789210200040002990010000008210000880a07fff0280000701000000030000c182106188c227bfbc1080000501000000030000c182106190c227bfbc400029c1010000008210000880a0600012800005010000000300010982106178c0204000c207a048c2004000a0100001c207a048c200400090100001400039d1010000008410000890100010030000c19210617094100002400039e801000000c207a048c200400090100001d207bfbc40002b6901000000c207bfbc9010200f9210000194102000961020009810200040002a3f010000009010200040002a9c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [58]:
X,label_y=vectorize()

/tmp/ipykernel_519201/3496483546.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  label = pd.read_csv('/media/dataset/dataset.csv', encoding = 'big5', dtype={'Column2': str})


...


In [59]:
label_y

['Mirai',
 'Unknown',
 'Android',
 'Mirai',
 'Bashlite',
 'Bashlite',
 'Mirai',
 'Bashlite',
 'Unknown',
 'Android',
 'Mirai',
 'Bashlite',
 'Mirai',
 'Dofloo',
 'Mirai',
 'Mirai',
 'Mirai',
 'Mirai',
 'Mirai',
 'Dofloo',
 'Unknown',
 'Mirai',
 'Mirai',
 'Unknown',
 'Unknown',
 'Mirai',
 'Bashlite',
 'Mirai',
 'Mirai',
 'Bashlite',
 'Bashlite',
 'Android',
 'Bashlite',
 'Bashlite',
 'Mirai',
 'Mirai',
 'Mirai',
 'Mirai',
 'Android',
 'Bashlite',
 'Mirai',
 'Android',
 'Bashlite',
 'Mirai',
 'Mirai',
 'Mirai',
 'Mirai',
 'Mirai',
 'Mirai',
 'Bashlite',
 'Mirai',
 'Bashlite',
 'Unknown',
 'Unknown',
 'Mirai',
 'Unknown',
 'Unknown',
 'Bashlite',
 'Xorddos',
 'Mirai',
 'Android',
 'Bashlite',
 'Unknown',
 'Bashlite',
 'Bashlite',
 'Bashlite',
 'Bashlite',
 'Mirai',
 'Mirai',
 'Android',
 'Mirai',
 'Mirai',
 'Bashlite',
 'Mirai',
 'Mirai',
 'Mirai',
 'Bashlite',
 'Mirai',
 'Mirai',
 'Bashlite',
 'Bashlite',
 'Unknown',
 'Mirai',
 'Android',
 'Bashlite',
 'Bashlite',
 'Bashlite',
 'Mirai',


In [8]:
print(type(X))
print(type(label_y))

<class 'scipy.sparse._csr.csr_matrix'>
<class 'list'>


In [3]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
def model():
    svm_classifier = LinearSVC(C=0.2)
    knn_classifier = KNeighborsClassifier(n_neighbors=3)
    nb_classifier =  MultinomialNB()
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=1000)
    return svm_classifier,knn_classifier,nb_classifier

In [4]:
svm_classifier,knn_classifier,nb_classifier=model()

In [5]:
type(knn_classifier)

sklearn.neighbors._classification.KNeighborsClassifier

In [51]:

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
def predict(svm_classifier,knn_classifier,nb_classifier,X,label_y):
    x_train, x_test, y_train, y_test = train_test_split(X,label_y,test_size = 0.2,shuffle = True)
    svm_classifier.fit(x_train,y_train)
    print("SVM:",accuracy_score(y_test,svm_classifier.predict(x_test)))
    knn_classifier.fit(x_train,y_train)
    print("KNN:",accuracy_score(y_test,knn_classifier.predict(x_test)))
    nb_classifier.fit(x_train,y_train)
    print("NB:",accuracy_score(y_test,nb_classifier.predict(x_test)))
#    mlp_classifier.fit(x_train,y_train)
#    print("MLP:",accuracy_score(y_test,mlp_classifier.predict(x_test)))

In [54]:
predict(svm_classifier,knn_classifier,nb_classifier,X,label_y)

/home/mickey/Tools/anaconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM: 0.6554621848739496
KNN: 0.7983193277310925
NB: 0.7647058823529411


/home/mickey/Tools/anaconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [56]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
svm_classifier1 = SVC(C=0.2)
svm_classifier2 = LinearSVC(C=0.2)
svm_classifier4 = SGDClassifier(loss='hinge', alpha=1/0.2)
x_train, x_test, y_train, y_test = train_test_split(X,label_y,test_size = 0.2,shuffle = True)
svm_classifier1.fit(x_train,y_train)
print("SVM1:",accuracy_score(y_test,svm_classifier1.predict(x_test)))
svm_classifier2.fit(x_train,y_train)
print("SVM2:",accuracy_score(y_test,svm_classifier2.predict(x_test))) #95.63

svm_classifier4.fit(x_train,y_train)
print("SVM4:",accuracy_score(y_test,svm_classifier4.predict(x_test))) #74.86

SVM1: 0.5210084033613446


/home/mickey/Tools/anaconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM2: 0.680672268907563
SVM4: 0.5798319327731093


/home/mickey/Tools/anaconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [1]:
from detector import subDetector
from sklearn.model_selection import train_test_split
d=subDetector()
d.extractFeature('/media/dataset/linuxmal/f8/')
X,label_y=d.vectorize()
x_train, x_test, y_train, y_test = train_test_split(X,label_y,test_size = 0.2,shuffle = True)

WARN: Relocs has not been applied. Please use `-e bin.relocs.apply=true` or `-e bin.cache=true` next time
INFO: Analyze all flags starting with sym. and entry0 (aa)
INFO: Analyze imports (af@@@i)
INFO: Analyze entrypoint (af@ entry0)
INFO: Analyze symbols (af@@@s)
INFO: Recovering variables
INFO: Analyze all functions arguments/locals (afva@@@F)
INFO: Analyze function calls (aac)
INFO: Analyze len bytes of instructions for references (aar)
INFO: Finding and parsing C++ vtables (avrr)
INFO: Analyzing methods
INFO: Finding xrefs in noncode section (e anal.in=io.maps.x)
INFO: Emulate functions to find computed references (aaef)
INFO: Recovering local variables (afva)
INFO: Type matching analysis for all functions (aaft)
INFO: Propagate noreturn information (aanr)
INFO: Integrate dwarf function information
INFO: Use -AA or aaaa to perform additional experimental analysis
WARN: Relocs has not been applied. Please use `-e bin.relocs.apply=true` or `-e bin.cache=true` next time
INFO: Analyze 

In [2]:
svm_classifier,knn_classifier,nb_classifier = d.pmodel(x_train,y_train)

/home/mickey/Tools/anaconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mickey/Tools/anaconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [3]:
d.predict(svm_classifier,knn_classifier,nb_classifier,x_test,y_test)

SVM: 0.7226890756302521
KNN: 0.7815126050420168
NB: 0.7563025210084033
